### a series of functions to help in the ground cover forecasting project

example use:

year_init = 1990

year_end = 2020

lat= -33.34392

lon= 145.28401

get_fc_bawap(year_init, year_end, lat, lon)


In [18]:
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import xarray as xr
import rasterio
import os
import time

In [19]:
def bawap_monthly_fname(year, var):
    
    #path = '/OSM/CBR/CoRE/working/timeseries/Climate/bawap/nc/'+var+'/month/'
    path = '/datasets/work/lw-bawapsilo/work/home/bawap/nc/'+var+'/month/'
    
    text_files = [f for f in os.listdir(path) if f.endswith(str(year)+'1231.nc')]
    
    return path+text_files[0] 

def get_bawap_pixel(year_init, year_end, lat, lon, var):
    var_month = var+'_month'
    df = pd.DataFrame()
    for year in range(year_init, year_end+1):
        fname = bawap_monthly_fname(year, var)
        bawap=xr.open_dataset(fname)
        pixel=bawap[var_month].sel(latitude=[lat], longitude=[lon], method='nearest').to_dataset().squeeze().to_dataframe().drop(columns=['latitude','longitude'])
        
        # remove the time from the date (needed because vph came with time (9am or 3pm)
        pixel.index = pixel.index.date
        df = df.append(pixel)
    
    return df

def get_bawap_pixel_all(year_init, year_end, lat, lon):
    rain = get_bawap_pixel(year_init, year_end, lat, lon, 'rain')
    tmax = get_bawap_pixel(year_init, year_end, lat, lon, 'tmax')
    tmin = get_bawap_pixel(year_init, year_end, lat, lon, 'tmin')
    rad  = get_bawap_pixel(year_init, year_end, lat, lon, 'rad')
    vph09  = get_bawap_pixel(year_init, year_end, lat, lon, 'vph09')
    vph15  = get_bawap_pixel(year_init, year_end, lat, lon, 'vph15')
                             
    df=pd.concat([rain,tmax,tmin,rad,vph09,vph15], axis=1)
        
    return df

def fc_monthly_fname(year, month):
    path='/datasets/work/lw-lpdaac/work/fc-wron/data/public/v310/australia/monthly/cover/'
    fname=path+'FC.v310.MCD43A4.A'+str(year)+'.'+'{:02d}'.format(month)+'.aust.006.tif'
    return fname

def fc_monthly_TotCov_fname(year, month):
    path='/datasets/work/lw-lpdaac/work/fc-wron/data/public/v310/australia/monthly/cover/'
    fname=path+'FC.v310.MCD43A4.A'+str(year)+'.'+'{:02d}'.format(month)+'.aust.006.Tot.tif'
    return fname

def get_fc_pixel(year_init, year_end, lat, lon):
    df_all=pd.DataFrame()
    for year in range(year_init, year_end+1):
        print('getting FC for year', year)
        for month in range(1,13):
            fname = fc_monthly_fname(year, month)
            if os.path.isfile(fname):
                raster = rasterio.open(fname)
                py, px = raster.index(lon, lat)
                window = rasterio.windows.Window(px, py, 1, 1)
                pix = raster.read(window=window)
                #print(pix)
                raster.close()
            else:
                pix=np.array([[[np.nan]],[[np.nan]],[[np.nan]]])

            df = pd.DataFrame()
            df['date']=[pd.to_datetime(str(year)+'{:02d}'.format(month), format='%Y%m')]
            df['PV'] = [pix.item(1)]
            df['NPV'] = [pix.item(2)]
            df['BS'] = [pix.item(0)]
            df.index=df['date']
            df=df.drop(columns='date')
            
            df_all=df_all.append(df)
            
    df_all[['PV','NPV','BS']] = df_all[['PV','NPV','BS']].replace(255, np.nan)
    df_all['TOT'] = df_all['PV']+df_all['NPV']
    df_all['date'] = pd.to_datetime(df_all.index)
    #df_all['MONTH']  = df_all.date.dt.month
    #df_all['YEAR']   =df_all.date.dt.year
    df_all = df_all.drop(columns='date')
    
    return df_all

def get_fc_bawap(year_init, year_end, lat, lon):
    fc = get_fc_pixel(year_init, year_end, lat, lon)
    bawap = get_bawap_pixel_all(year_init, year_end, lat, lon)
    
    return pd.concat([bawap,fc], axis=1)

def get_fc_bawap_rain(year_init, year_end, lat, lon):
    fc = get_fc_pixel(year_init, year_end, lat, lon)
    bawap = get_bawap_pixel(year_init, year_end, lat, lon, 'rain')
    
    return pd.concat([bawap,fc], axis=1)

def open_TotCov_xarray(year, month):
    fname = fc_monthly_TotCov_fname(year, month)
    if os.path.isfile(fname):
        da = xr.open_rasterio(fc_monthly_TotCov_fname(year, month))
        da['time'] = pd.to_datetime(str(year)+'-'+str(month))
        return da

    


In [20]:
def get_fc_box(year_init, year_end, left, right, top, bottom):
    first=True
     
    for year in range(year_init, year_end+1):
        print('getting FC for year', year)
        for month in range(1,13):
            fname = fc_monthly_fname(year, month)
            if os.path.isfile(fname):
                da = xr.open_rasterio(fname)
                da = da.sel(x=slice(left,right), y=slice(top,bottom)) 
                da = da.assign_coords(time=pd.to_datetime(str(year)+'-'+str(month)))
                if first:
                    da_master=da
                    first=False
                else:
                    da_master=xr.concat([da_master,da], dim='time')   
        
    da_master = da_master.to_dataset('band')
    da_master = da_master.rename({2: 'PV', 3: 'NPV', 1:'BS'}) 
                
    return da_master


def get_bawap_box(year_init, year_end, left, right, top, bottom, var):
    first=True
    var_month = var+'_month'
    df = pd.DataFrame()
    for year in range(year_init, year_end+1):
        print(year)
        fname = bawap_monthly_fname(year, var)
        bawap=xr.open_dataset(fname)
        bawap=bawap[var_month].sel(latitude=slice(top,bottom), longitude=slice(left,right))         
        if first:
            bawap_master=bawap
            first=False
        else:
            bawap_master=xr.concat([bawap_master,bawap], dim='time')       
    return bawap_master


In [21]:
#year_init = 2019
#year_end = 2019
#month = 2
#top=-35
#left = 145
#bottom=-35.2
#right = 145.2
#var='rain'
#bawap=get_bawap_box(2000,2020, left, right, top, bottom, 'rain')
#fc =get_fc_box(year_init, year_end, left, right, top, bottom)
#bawap, fc

In [22]:
#bawap_monthly_fname(2020, 'rain')
#get_bawap_pixel_all(2020, 2021, -35, 145)

In [23]:
#data= get_fc_bawap_rain(1990, 2021,-12.481585, 134.022019)
#data

In [24]:
#data[['PV','NPV','BS','TOT','rain_month']].plot(figsize=[25,5], marker='o')


In [60]:
#data[['PV','NPV','BS','TOT','rain_month']].interpolate(limit_area='inside').plot(figsize=[25,5], marker='o')

In [31]:
#fc = get_fc_box(2001, 2020, 145, 146, -35, -36)


In [32]:
#fc = fc.where(fc < 250)
#fc

In [33]:
#fc['TOT'] = fc['PV'] + fc['NPV']